In [ ]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import json
import cProfile
import re
import time

In [ ]:
%run GetLossObj.ipynb

In [ ]:
baseDir = "../save/local/models/model40/"
# expData = torch.load(baseDir+"experimentData.pth")
# plotTrainReconLosses = torch.load(baseDir + "plot_train_recon_losses", map_location="cpu")
# plotValReconLosses = torch.load(baseDir + "plot_val_recon_losses", map_location="cpu")
# plotTrainKldLosses = torch.load(baseDir + "plot_train_KLD_losses", map_location="cpu")
# plotValKldLosses = torch.load(baseDir + "plot_val_KLD_losses", map_location="cpu")
with open(baseDir + "args.txt") as f:
    args = f.read()
args = Bunch(json.loads(args))
args.cuda = False
args.no_cuda = True
args._device = "CPU"
args.dropout= 0.0
if args.dataset == "traffic":
    args.output_dim = args.x_dim
else: #args.dataset == "human":
    args.output_dim = args.x_dim * 2

In [ ]:
print(args)

In [ ]:
print(args.initial_lr, args.dataset)

In [ ]:
dataDir = "../../data/Traffic/Processed/trafficWithTime/"



In [ ]:
dataDict = getDataLoaders(dataDir)

In [ ]:
def getPredictions(Pi, Mu, Sigma, reps=100):
    preds = []
    for i in range(reps):
        pred = getlossobj.model.generatePred(Pi, Mu, Sigma, individual=True)
        preds.append(getlossobj.dataDict["scaler"].inverse_transform(pred, permute=False))
    predsStack = torch.stack(preds, dim=0)
    mean = torch.mean(predsStack,dim=0)
    std = torch.std(predsStack, dim=0)
    return mean, std
    

In [ ]:
def plotRandomInstance(mean, std,targets, datas, dataset, instance, sensor):
    t = targets[instance, :, sensor]
    d = datas[instance, :, sensor]
    plt.plot(range(5,65, 5),mean, label="Pred")
    plt.fill_between(range(5,65, 5),mean-1.96*std, mean+1.96*std, alpha=0.25)
    plt.plot(range(5,65, 5),t, label="target")
    plt.plot(range(-55,5, 5), d, label="input")
    plt.legend()
    plt.xlabel("Minutes relative to prediction time")
    plt.ylabel("Speed (mph)")
    plt.title("random sensor sample {}".format(dataset))
    plt.show()

In [ ]:
getlossobj = PostProcess(baseDir,args, dataDict=dataDict, chooseModel="sketch-rnn")
getlossobj.prep("sketch-rnn_full_model.pth", "train", 0)

In [ ]:
(inputData, target) = next(map(dataDict["scaler"].transformBatchForEpoch, dataDict["train"]))
indexIntoDir += 1
output = getlossobj.model(inputData, target, 0)
Pi, Mu, Sigma, latentMean, latentStd = output
Pi = Pi.cpu().detach()
Mu = Mu.cpu().detach()
Sigma = Sigma.cpu().detach()
latentMean = latentMean.cpu().detach()
latentStd = latentStd.cpu().detach()
instance = np.random.randint(Pi.shape[1])
sensor = np.random.randint(Pi.shape[2])
# Pi = Pi[:,instance, sensor,:]
# Mu = Mu[:,instance, sensor,:]
# Sigma = Sigma[:,instance, sensor,:]
target = dataDict["scaler"].inverse_transform(target)
inputData = dataDict["scaler"].inverse_transform(inputData[:,:,0,:])
mean, std = getPredictions(Pi[:,instance, sensor,:], Mu[:,instance, sensor,:], Sigma[:,instance, sensor,:])
plotRandomInstance(mean.numpy(), std.numpy(), target.numpy(), inputData.numpy(),"train", instance, sensor)

In [ ]:
trainRetVals = getlossobj.getLossAtEpoch("sketch-rnn_full_model.pth", "train")


In [ ]:
valRetVals = getlossobj.getLossAtEpoch("sketch-rnn_full_model.pth", "val")

In [ ]:
testRetVals = getlossobj.getLossAtEpoch("sketch-rnn_full_model.pth", "test")

In [ ]:
print("train: {:.4f}, val: {:.4f}, test: {:.4f}".format(trainRetVals["reconLoss"],valRetVals["reconLoss"], testRetVals["reconLoss"]))

In [ ]:
inputData.shape

In [ ]:
instance = np.random.randint(trainRetVals["datas"].shape[0])
sensor = np.random.randint(trainRetVals["datas"].shape[2])
PiT = torch.from_numpy(trainRetVals["Pi"][:,instance, sensor,:])
MuT = torch.from_numpy(trainRetVals["Mu"][:,instance, sensor,:])
SigmaT = torch.from_numpy(trainRetVals["Sigma"][:,instance, sensor,:])
mean, std = getPredictions(PiT, MuT, SigmaT, reps=100)
plotRandomInstance(mean.data.numpy(), std.data.numpy(), trainRetVals["targets"], trainRetVals["datas"], "Train", instance, sensor)

In [ ]:
instance = np.random.randint(valRetVals["datas"].shape[0])
sensor = np.random.randint(valRetVals["datas"].shape[2])
PiT = torch.from_numpy(valRetVals["Pi"][:,instance, sensor,:])
MuT = torch.from_numpy(valRetVals["Mu"][:,instance, sensor,:])
SigmaT = torch.from_numpy(valRetVals["Sigma"][:,instance, sensor,:])
mean, std = getPredictions(PiT, MuT, SigmaT, reps=100)
plotRandomInstance(mean.data.numpy(), std.data.numpy(), valRetVals["targets"], valRetVals["datas"], "Validation", instance, sensor)

In [ ]:
instance = np.random.randint(testRetVals["datas"].shape[0])
sensor = np.random.randint(testRetVals["datas"].shape[2])
PiT = torch.from_numpy(testRetVals["Pi"][:,instance, sensor,:])
MuT = torch.from_numpy(testRetVals["Mu"][:,instance, sensor,:])
SigmaT = torch.from_numpy(testRetVals["Sigma"][:,instance, sensor,:])
mean, std = getPredictions(PiT, MuT, SigmaT, reps=100)
plotRandomInstance(mean.data.numpy(), std.data.numpy(), testRetVals["targets"], testRetVals["datas"], "Test", instance, sensor)

In [ ]:
total = 0
for p in getlossobj.model.parameters():
    total += np.prod(p.shape)
print(total)

In [ ]:
plotTrainValCurve(plotTrainReconLosses, plotValReconLosses, plotTrainKldLosses, plotValKldLosses)

In [ ]:
plotTrainValCurve(np.array(plotTrainReconLosses) + np.array(plotTrainKldLosses), np.array(plotValReconLosses) + np.array(plotValKldLosses), labelTotal=True)

In [ ]:
next(iter(getlossobj.model.parameters())).is_cuda

In [ ]:
for name, p in getlossobj.model.named_parameters():
    if bias in 
    print(name)